# IMPORTAÇÕES

In [15]:
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import unidecode
import Database
import json
from datetime import datetime

# Nome da Tabela

In [16]:
table_name = 'pareamentos2'

## Pegando Dados

In [17]:
db = Database.Database('CNPq')
engine = Carga.db_engine()

In [18]:
pareados = pd.read_sql(table_name, engine)
pareados.size
#size: 138.303

138303

In [19]:
#Carregar arquivo na memória

dt = pd.read_sql(f"select * from public.indicadores_pareamento where not ano_doutorado is null and id in (SELECT distinct(unnest(array[id, pareado_1, pareado_2, pareado_3])) AS id FROM public.{table_name})", engine)
dt.size
#size: 16.357.924    


11522214

## Normalizando Dados

In [20]:
## Retirada dos não doutores - Existem 3 não doutores contemplados!!!
dt = dt[pd.notnull(dt.ano_doutorado)]


## Normalizando as Tabelas

#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)

#Preenchendo Nan com "nenhum" nas áreas e vínculos
dt.area_demanda_bruta = dt.area_demanda_bruta.fillna('nenhum')
dt.areas_conhecimento = dt.areas_conhecimento.fillna('nenhum')
dt.tipos_vinculo = dt.tipos_vinculo.fillna('nenhum')
dt.enquadramento_vinculo = dt.enquadramento_vinculo.fillna('nenhum')

#Removendo acentos e caracteres especiais. Colocando tudo em minúsculas.
dt.area_demanda_bruta = dt.area_demanda_bruta.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.areas_conhecimento = dt.areas_conhecimento.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.tipos_vinculo = dt.tipos_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.enquadramento_vinculo = dt.enquadramento_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()              

## Criando tabelas parciais
financiados = dt[~pd.isnull(dt.pgtos)]
dt = dt[pd.isnull(dt.pgtos)]


## Alguns pesquisadores foram financiados em um ano, mas não em outro. Assim, aparece duas vezes, uma com pgtos=Nan, outra com pgtos > 0.
## Assim, é necessário retirar da lista de pareamento aqueles que foram financiados em algum momento.
## Da mesma forma, com certeza houveram aqueles financiados mais de um ano.
dt = dt[~dt.id.isin(financiados.id.unique())] #Percebi alguns erros, então tive a certeza de remover os financiados da tabela de pareamento



## Criando Excel Writer

In [21]:
path = f"C:/Users/silva/CNPq/Lattes/Tabela_Melaine_{table_name}.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

# Pegando tabela com contagem de grupos

In [22]:
lista_indicadores = []
for grupo in dt.grupo.unique():
    dt_grupo = dt[dt.grupo == grupo]
    financiados_grupo = financiados[financiados.grupo == grupo]
    tabela = {
        'grupo': grupo,
        'financiados_média': financiados_grupo.qty_2012.mean(),
        'pareados_média': dt_grupo.qty_2012.mean(),
        'financiados_desvio_padrão': financiados_grupo.qty_2012.std(),
        'pareados_desvio_padrão': dt_grupo.qty_2012.std()
    }
    lista_indicadores.append (tabela)



## Tabela Ano Doutorado

In [23]:
financiados_dt = financiados[['id', 'ano_doutorado']].drop_duplicates() 
dt_dt = dt[['id', 'ano_doutorado']].drop_duplicates()  
tabela = {
    'grupo': 'ano_doutorado',
    'financiados_média': financiados_dt.ano_doutorado.mean(),
    'pareados_média': dt_dt.ano_doutorado.mean(),
    'financiados_desvio_padrão': financiados_dt.ano_doutorado.std(),
    'pareados_desvio_padrão': dt_dt.ano_doutorado.std()
}
lista_indicadores.append (tabela)
print(tabela)

{'grupo': 'ano_doutorado', 'financiados_média': 2003.1389485405234, 'pareados_média': 2009.1114151086792, 'financiados_desvio_padrão': 7.9930880939754365, 'pareados_desvio_padrão': 9.316022975881928}


### Salvando a primeira planilha no Writer

In [24]:
tabela = pd.DataFrame(lista_indicadores)
tabela.to_excel(writer, sheet_name = 'Grupos')

## Tabela Sexo

In [25]:
financiados_sexo = financiados[['id', 'sexo']].drop_duplicates() 
dt_sexo = dt[['id', 'sexo']].drop_duplicates()  


In [26]:
fsm = financiados_sexo[financiados_sexo.sexo == 'M'].id.count()
fsf = financiados_sexo[financiados_sexo.sexo == 'F'].id.count()
dsm = dt_sexo[dt_sexo.sexo == 'M'].id.count()
dsf = dt_sexo[dt_sexo.sexo == 'F'].id.count()

In [27]:
tabela = [['Descrição', 'Valor']]
tabela.append(['grupo', 'sexo'])
tabela.append(['financiados: homens %', fsm/(fsm + fsf)])
tabela.append(['pareados: homens %', dsm/(dsm + dsf)])
tabela.append(['total_financiados', financiados_sexo.id.count()])
tabela.append(['total_pareados', dt_sexo.id.count()])
sexo = pd.DataFrame(tabela)
print(sexo)               


                       0         1
0              Descrição     Valor
1                  grupo      sexo
2  financiados: homens %  0.590593
3     pareados: homens %  0.553056
4      total_financiados     12573
5         total_pareados      6579


In [28]:
sexo.to_excel(writer, sheet_name = 'Sexo')


## Salvando o arquivo

In [29]:
writer.save()
writer.close()

C:\Python\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
